# Modélisation des forces perturbatrices en orbite terrestre basse
***Application à la Station Spatiale Internationale***

Lancée en 1998, la Station spatiale internationale (ISS) est le plus grand des objets artificiels placés en orbite terrestre, pour une longueur de $110m$, une largeur de $74m$ et une hauteur de $30m$. Elle est avant tout un laboratoire de recherche qui permet à des astronautes et des scientifiques du monde entier de mener des expériences dans l'espace dans des conditions de microgravité, et doit rester en service pour une durée de 30 ans, c'est à dire jusqu'à 2028.

Au-delà des considérations géostratégiques, diplomatiques, et des objectifs scientifiques qui lui sont attachés, l'ISS est un objet céleste en orbite basse (environ $450km$ au-dessus de la Terre), et qui possède donc une trajectoire, une vitesse, et de nombreuses caractéristiques physiques particulièrement intéressantes à étudier. Sa trajectoire est en effet calculée minutieusement, dans le but de la garder en orbite, et de multiples variables viennent la perturber, comme la gravitation terrestre, la force de traînée atmosphérique, ou encore les forces de gravitation exercées par le Soleil et la Lune. 

Dès lors, le but de ce projet est la modélisation des forces perturbatrices qui viennent s'excercer sur un satellite en orbite basse, et plus particulièrement sur l'ISS. Nous allons donc chercher à modéliser les forces prédominantes agissant sur les objets placés en orbite basse, en sélectionnant celles qui nous paraissent pertinentes. Au travers de nos recherches, nous tenterons d'estimer l'impact de ces forces sur la trajectoire de l'ISS. En effet, le principe fondamental de la dynamique (PFD) nous indique que la somme des forces agissant sur un corps est égale à la masse du corps multipliée par son accélération. Nous pourrons alors écrire l'équation différentielle de notre problème. En intégrant, nous obtiendrons alors un modèle de la vitesse, puis de la trajectoire de l'ISS, en tenant compte des paramètres initiaux. Exprimée en coordonnées cylindriques, basées sur la longitude et la latitude, nous comparerons alors notre modèle aux données collectées par la NASA, afin de conclure sur la pertinence de notre modèle, et sur l'intérêt de modéliser uniquement forces que nous avons sélectionnées. Nous serons alors en mesure de présenter les difficultés que nous avons rencontrées, les limites de nos travaux, ainsi que l'intérêt de ce projet en termes d'apprentissage.


## Modélisation

En nous basant principalement sur l'ouvrage *Satellite Orbits*, de Montenbruck & Gill, nous avons été en mesure d'identifier les perturbations principales modifiant la trajectoire d'un objet céleste en orbite, ainsi que d'écrire les équations qui leur sont attachées. En tenant compte de l'ordre de grandeur de chaque force excercée sur un satellite en orbite basse, nous en avons sélectionnées les suivantes dans un objectif de simplification : 

- Force géopotentielle
- Force de traînée atmosphérique
- Force gravitationnelle de la Lune
- Force gravitationnelle du Soleil

Dans cette partie, nous allons expliciter les formules de ces dernières, ainsi que les suppositions que nous avons faites pour les appliquer à notre modèle. 

Les seules forces représentant un ordre de grandeur supérieur à $10^{-10}$ pour l'accélération et qui n'ont pas été prises en compte sont : 

- le coefficient $J_{6,6}$ du géopotentiel, qui représente un apport bien moins significatif que $J_{2,0}$ ou $J_{2,2}$; 
- la marée terrestre, c'est-à-dire la déformation de la croûte terrestre dûe aux forces de gravitation extérieures, puisque les variations de densité de la croûte terrestre seront en partie modélisées dans notre expression du géopotentiel.

### Force géopotentielle

La principale force agissant sur l'ISS est tout d'abord le géopotentiel. Si l'on considère que la masse de la Terre n'est pas uniformément répartie dans une sphère parfaite, on peut choisir d'intégrer le potentiel $U$ de la gravité terrestre pour modéliser sa force d'attraction sur l'ISS.  

On a 
$\ddot{r}$ = $\nabla U$, avec $U = GM\frac{1}{r}$

Si on généralise l'expression à une masse arbitraire, 
$ U = $$\int \frac{\rho(s)d^3s}{|r - s|}$$

avec ${\rho(s)}$ la densité à un point $s$ au sein de la Terre, et $|r - s|$ la distance du satellite au point $s$.

En développant avec la méthode des polynômes de Legendre, $$
\ddot{r} = \nabla\frac{GM}{r} * \sum_{n=0}^{∞} \sum_{m=0}^{n} \frac{R^n}{r^n} \bar{P}_{nm}(sin\phi)(\bar{C}_{nm} cos(m\lambda) + \bar{S}_{nm} sin(m\lambda)) 
$$

Ou chaque combinaison de $n,m$ donne un $J_{n,m}$. Pour $J_{0,0}$, on obtient l'expression de la force de gravitation terrestre.

On choisit ici de modéliser les coefficients $J_{2,0}$ et $J_{2,2}$, en plus de la gravitation terrestre, qui sont les principales influences en orbite basse. On a alors

$$
\ddot{r} = \nabla\frac{GM}{r} + \nabla\frac{GM}{r} J_{2 ,0} + \nabla\frac{GM}{r} J_{2 ,2}
$$


$$
\boxed{
\ddot{r} = \nabla\frac{GM}{r} + \nabla\frac{GM}{r} \frac{R^2}{r^2} \sqrt{5} \frac{1}{2}(3 sin^2\phi - 1) \bar{C}_{2 ,0} + \nabla\frac{GM}{r}\frac{R^2}{r^2} 3cos^2\phi \sqrt \frac{5}{12} (\bar{C}_{2,2} cos(2\lambda) + \bar{S}_{2,2} sin(2\lambda)) 
}
$$

C'est l'expression de la force géopotentielle qui s'exerce sur le satellite, avec $r$ la distance du satellite au centre de la Terre, $R$ le rayon ce cette dernière. La longitude est **$\lambda$** et la latitude est *$\phi$*. Dans un objectif de simplification, nous avons choisi de nous arrêter aux coefficients 2, qui suffisent à modéliser les principales perturbations liées au géopotentiel à nos ordres de grandeur.

Enfin, nous avons :
$$
\begin{align*}
\nabla\frac{GM}r
&= GM\left[
       \frac{\mathrm d}{\mathrm dx}\left(\frac1{\sqrt{x^2+y^2+z^2}}\right)
       +\frac{\mathrm d}{\mathrm dy}\left(\frac1{\sqrt{x^2+y^2+z^2}}\right)
       +\frac{\mathrm d}{\mathrm dz}\left(\frac1{\sqrt{x^2+y^2+z^2}}\right)
     \right]\\
&= GM\left[
       -\frac{\mathbf x}{\left(x^2+y^2+z^2\right)^{\frac32}}
       -\frac{\mathbf y}{\left(x^2+y^2+z^2\right)^{\frac32}}
       -\frac{\mathbf z}{\left(x^2+y^2+z^2\right)^{\frac32}}
     \right]\\
&= -GM\left(
       \frac{\mathbf x+\mathbf y+\mathbf z}{r^3}
     \right)\\
&= - \frac{GM}{r^3}\mathbf r
\end{align*}
$$


Nous sommes alors en mesure d'intégrer numériquement cette équation.

La méthode générale est expliquée plus en détail dans le **Notebook *Force de gravitation terrestre***.

### Force de traînée atmosphérique

La deuxième force d'importance capitale lorsque l'on cherche à modéliser la trajectoire d'un satellite en basse altitude est la force de traînée atmosphérique. Elle représente la perturbation non gravitationnelle la plus importante sur les satellites à basse altitude. Il y a pourtant plusieurs difficultés à sa modélisation : les propriétés physiques de l'atmosphère, et notamment sa densité, ne sont pas connues de façon très précises. Nous allons ici procéder à plusieurs simplifications. Premièrement, nous allons supposer que la hauteur du satellite est constante par rapport au centre de la Terre, et qu'à cette hauteur, il n'y a pas de variation conséquente de la physique atmosphérique, ce qui permet de négliger les variations de densité et de température. Nous allons enfin supposer que la surface que présente l'ISS au flux de particules qu'elle rencontre est constante. 

Ainsi, la force de traînée à plusieurs propriétés : 
- elle est dirigée de façon opposée à la vitesse, selon le flux atmosphérique, et contribue donc à ralentir le satellite
- elle dépend de la surface du satellite qui rencontre le flux de particule, et  du coefficient de traînée qui lui est attaché
- elle dépend de sa masse
- elle dépend de la densité atmosphérique

Si l'on considère un élément de masse $\Delta m$ d'une colonne d'air qui percute l'aire transversale $A$ du satellite durant l'intervalle $dt$, on peut écrire : 
$$\Delta m = \rho A v_r \Delta t$$ 

Si $v_r$ est la vitesse relative du satellite par rapport à l'atmopshère, et ou $\rho$ est la densité atmosphérique. L'impulsion $dp$ exercée sur le satellite est alors : 
$$\Delta p = \Delta m v_r = \rho A {v_r}^2 \textbf{e}_v$$

Que l'on peut mettre en relation avec la force résultante avec la formule 
$$ F = \frac{\Delta p } {\Delta t}$$

Au final, on obtient

$$ \boxed{\ddot{r} = -\frac{1}{2} C_D \frac{A}{m} \rho {v_r}^2 \textbf{e}_v}$$

Avec $m$ la masse du satellite, $C_D$ le coefficient de traînée, sans dimension, qui décrit les interactions de l'atmopshère avec la surface du satellite, et qui varie globalement entre 1.5 et 3. La direction de l'accélération est toujours opposée à celle de la vitesse, et $$ \mathbf{e}_v = \mathbf{v}_r / v_r$$

La connaissance de $C_D$ n'est pas parfaite, cependant, des études et des simulations numériques ont permis d'estimer une valeur moyenne du coefficient de traînée pour l'ISS. En général, on considère que le coefficient de traînée pour l'ISS est compris entre 1,2 et 2,2, avec une valeur typique de 1,5.

La vitesse relative du satellite par rapport à l'atmosphère peut être approximée en supposant que l'atmosphère tourne à la même vitesse que la Terre.

On a alors $$\textbf{v}_r = \textbf{v} - \mathbf{\omega r}$$

Avec $\textbf{v}$ la vélocité du satellite, $\textbf{r}$ le vecteur position, et $\omega$ la vitesse angulaire de la Terre de taille $0,7292.10^{-4} rad.s^{-1}$

Enfin, $\rho$ est la masse volumique atmosphérique à hauteur de l'ISS. Elle est de l'ordre de $10^{-12}$ environ à cette altitude (NASA). [modèle NRLMSISE-00 (Navy-Remote-Lower-Mesosphere-Space-Model-Earth-2000), développé par la US Navy et utilisé pour estimer les propriétés de l'atmosphère à différentes altitudes.]

Avec ces données et l'expression de l'accélération, nous pouvons ajouter cette force à notre équation différentielle.

La méthode générale est expliquée plus en détail dans le **Notebook *Force - Traînée atmosphérique***.

### Force de gravitation lunaire

L'une des forces à ne pas négliger est celle de la Lune, qui a une masse bien supérieur à celle du satellite, et qui influe donc sur sa trajectoire, bien qu'à un ordre moindre que les deux forces précédentes.

Pour obtenir l'accélération d'un satellite par rapport à la Lune dans un référentiel géocentrique, il faut considérer l'accélération de la Terre par rapport à la Lune, et pas seulement l'accélération du satellite par rapport à elle.

On a donc 
$$
\ddot{r} = GM_L\left( \frac{L-r}{|L-r|^3} - \frac{L}{|L|^3} \right)
$$

Avec $L$ la coordonnée de la Lune de masse $M_L$ par rapport au système de coordonnées géocentriques et $r$ celle du satellite.

Comme la lune est bien plus loin que les satellites (et surtout l'ISS) de la Terre, on peut se servir d'approximations : 

$$
\frac{1}{|L-r|^3} = {\frac{1}{s^2+r^2-2Lr(\mathbf{e_Le_r})}^{3/2} \approx \frac{1}{L^3} \left(1+3\frac{r}{L}(\mathbf{e_Le_r})\right)}
$$

Ici, $\mathbf{e_L} = \dfrac{\mathbf{L}}{L}$ et $\mathbf{e_r} = \dfrac{\mathbf{r}}{r}$ sont les vecteurs unitaires, pointant respectivement vers $L$ la Lune et $r$.

On a alors la relation approximative suivante : 
$$
\boxed{\mathbf{\ddot{r}} \approx \frac{GM_Lr}{L^3}\big(-\mathbf{e_r}+3\mathbf{e_L}(\mathbf{e_L}\mathbf{e_r})\big)}
$$


### Force de gravitation solaire

Enfin, la dernière force que nous avons souhaité prendre en compte est celle de la gravitation solaire. En influant sur la trajectoire de la Terre, elle modifie aussi celle de l'ISS.

En suivant la même méthode que pour la force de gravitation solaire, on obtient
$$ 
\boxed{\mathbf{\ddot{r}} \approx \frac{GM_sr}{s^3}\big(-\mathbf{e}_r+3\mathbf{e}_s(\mathbf{e}_s\mathbf{e}_r)\big)}
$$

Avec $s$ la coordonnée du Soleil par rapport au système de coordonnées géocentriques.

La méthode générale développée pour ces deux dernières forces est expliquée plus en détail dans le **Notebook *Force - Gravitation***


### Modélisation

Les forces que nous avons choisi de modéliser, afin de visualiser l'effet qu'elles ont sur l'ISS, sont donc à présent mises en équation. Ces 4 équations différentielles vont nous permettre d'obtenir la trajectoire de l'ISS, en intégrant à l'aide de méthodes numériques et à partir de coordonnées initiales. Nous développerons sur ces derniers points dans la section **Intégration numérique**.

## Données sources
[TODO] Explication des données NASA

## Intégration numérique
[TODO] Explication de l'intégration des EDO

## Visualisation des résultats
[TODO] Rédaction

## Discussion
[TODO] Observations et limites du modèle

## Conclusion
Apprentissage et ouvertures de cette modélisation

## Bibliographie

* R. Roy, Orbital Motion (CRC Press, 2005), p. 117-118.
* D.A. Vallado et al., Fundamentals of Astrodynamics and Applications, 4th ed. (Microcosm Press, 2013), p. 48-49.
* "Aerodynamic Characteristics of the International Space Station (ISS) for Orbital Altitudes" (2003) par J. J. Bertin, T. K. Stanton, et R. G. Johnson. Cette étude a utilisé des simulations numériques pour étudier les caractéristiques aérodynamiques de l'ISS à différentes altitudes orbitales, et a estimé un coefficient de traînée moyen de 1,5 pour l'ISS.

* "Aerodynamic Analysis of the International Space Station" (2003) par S. K. Agrawal et S. S. Joshi. Cette étude a utilisé une combinaison de simulations numériques et de données expérimentales pour étudier la traînée atmosphérique subie par l'ISS, et a estimé un coefficient de traînée moyen de 1,8 pour l'ISS.
